<a href="https://colab.research.google.com/github/MorgooN/ML-Labs/blob/main/LAB4_kNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Flatten, Rescaling, Concatenate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
!pip install split_folders

In [3]:
!gdown --id 1SK47fesS6Ek_XHVdjWCAmjgY95YyVz10

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1SK47fesS6Ek_XHVdjWCAmjgY95YyVz10
To: /content/lab4.zip
100% 614M/614M [00:04<00:00, 125MB/s] 


In [ ]:
!unzip lab4.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/unsavory/1223.jpg  
  inflating: train/unsavory/1224.jpg  
  inflating: train/unsavory/1225.jpg  
  inflating: train/unsavory/1226.jpg  
  inflating: train/unsavory/1227.jpg  
  inflating: train/unsavory/1228.jpg  
  inflating: train/unsavory/1229.jpg  
  inflating: train/unsavory/1230.jpg  
  inflating: train/unsavory/1231.jpg  
  inflating: train/unsavory/1232.jpg  
  inflating: train/unsavory/1233.jpg  
  inflating: train/unsavory/1234.jpg  
  inflating: train/unsavory/1235.jpg  
  inflating: train/unsavory/1236.jpg  
  inflating: train/unsavory/1237.jpg  
  inflating: train/unsavory/1238.jpg  
  inflating: train/unsavory/1239.jpg  
  inflating: train/unsavory/1240.jpg  
  inflating: train/unsavory/1241.jpg  
  inflating: train/unsavory/1242.jpg  
  inflating: train/unsavory/1243.jpg  
  inflating: train/unsavory/1244.jpg  
  inflating: train/unsavory/1245.jpg  
  inflating: train/unsavory/1246.jp

In [3]:
batch_size = 8
img_height = 112
img_width = 112

train_ds = tf.keras.utils.image_dataset_from_directory(
  directory='/content/train', # путь к папке с обучающими данными
  label_mode = 'int',       # представление правильных ответов картинок в виде чисел (Label Encoding): 0, 1, 2, 3... (Также можно использовать One-Hot Encoding, для этого напишите categorical) (Если у вас задача бинарной классификации пишите binary)
  color_mode='rgb',         # представление каждой картинки в RGB формате
  batch_size=batch_size,    # параметр, с помощью которого можно регулировать порцию подаваемых примеров (картинок) для сети за одну итерацию обучения
  seed=123,
  image_size=(img_height, img_width), # нейронная сеть всегда принимает изображение определённого размера. Поэтому необходимо предварительно поменять размер каждого изображения
  shuffle=True)                       # перетасовка данных, чтобы генерировать данные не в определённом порядке (сначала 1 класс, потом 2 класс), а случайным образом

test_ds = tf.keras.utils.image_dataset_from_directory(
  directory='/content/test',  # путь к папке с тестовыми данными
  label_mode = 'int',
  color_mode='rgb',
  batch_size=batch_size,
  seed=123,
  image_size=(img_height, img_width),
  shuffle=False)


normalization_layer = Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
normalized_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))



Found 11220 files belonging to 2 classes.
Found 600 files belonging to 2 classes.


In [4]:
# Функция формирования набора данных (картинки и метки)
def extract_dataset_images(dataset):
  labels = []
  images = []
  for batch, batch_labels in dataset:
    images.append(batch.numpy().reshape((batch.shape[0], -1))) # Делаем картинки в виде векторов
    labels.append(batch_labels.numpy())
  return np.concatenate(images), np.concatenate(labels)

x_train, y_train = extract_dataset_images(train_ds)

In [5]:
def train_and_test_KNN(x_train, y_train):
  # Обучение KNN на картинках (количество соседей n_neighbors, метрику расстояний и функцию взвешивания нужно подобрать)
  knn_classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', weights='distance')
  knn_classifier.fit(x_train, y_train)

  # Тестирование

  # Извлекаем тестовые картинки и соответствующие метки
  x_test, y_test = extract_dataset_images(test_ds)
  # Классификация тестовых изображений с использованием KNN
  y_pred = knn_classifier.predict(x_test)

  accuracy = accuracy_score(y_test, y_pred)
  print("KNN accuracy on test data:", accuracy)

train_and_test_KNN(x_train, y_train)

KNN accuracy on test data: 0.8066666666666666


In [6]:
x_train, y_train = extract_dataset_images(train_ds)

In [7]:
def train_and_test_PCA_KNN(x_train, y_train):
  # Применение PCA для извлечения только полезной информации и уменьшения размерности признаков (количество компонент нужно подобрать)
  pca = PCA(n_components=170)
  x_train_pca = pca.fit_transform(x_train)

  # Обучение KNN на сжатых признаках PCA (количество соседей n_neighbors, метрику расстояний и функцию взвешивания нужно подобрать)
  knn_classifier = KNeighborsClassifier(n_neighbors=9, metric='minkowski', weights='distance')
  knn_classifier.fit(x_train_pca, y_train)

  # Тестирование

  # Извлекаем тестовые картинки и соответствующие метки
  x_test, y_test = extract_dataset_images(test_ds)

  # Применение созданного ранее PCA для тестовых данных
  x_test_pca = pca.transform(x_test)

  # Классификация тестовых изображений с использованием KNN
  y_pred = knn_classifier.predict(x_test_pca)

  accuracy = accuracy_score(y_test, y_pred)
  print("PCA + KNN accuracy on test data:", accuracy)

train_and_test_PCA_KNN(x_train, y_train)

PCA + KNN accuracy on test data: 0.8166666666666667


In [8]:
def createPreTrainedModel(input_shape):

  base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

  # Выбор последнего слоя максимального пулинга для извлечения высокоуровневых признаков
  max_pooling_layer = base_model.get_layer('block5_pool').output

  # Применение Global Average Pooling (для усреднения признаков и таким образом уменьшения размерности признаков)
  global_avg_pooling_layer = GlobalAveragePooling2D()(max_pooling_layer)

  # Применение Flatten (чтобы выводить вектор признаков, поскольку KNN работает с векторами)
  flatten_layer = Flatten()(global_avg_pooling_layer)

  # Создание новой модели
  model = Model(inputs=base_model.input, outputs=flatten_layer)
  return model


def extract_features(dataset, model):
    features = []
    labels = []
    for batch_images, batch_labels in dataset:
        batch_features = model.predict(batch_images)
        labels.append(batch_labels.numpy())
        features.append(batch_features)
    return np.concatenate(features), np.concatenate(labels)

In [11]:

model = createPreTrainedModel((112, 112, 3))
x_train_features, y_train_labels = extract_features(train_ds, model)

1/1 [==============================] - 1s 540ms/step


In [13]:
def train_and_test_CNN_PCA_KNN(x_train_features, y_train_labels):
  # Применение PCA для извлечения только полезной информации и уменьшения размерности признаков (количество компонент нужно подобрать)
  pca = PCA(n_components=250)
  x_train_pca = pca.fit_transform(x_train_features)

  # Обучение KNN на уменьшенных признаках (количество соседей n_neighbors, метрику расстояний и функцию взвешивания нужно подобрать)
  # Список метрик: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.distance_metrics.html#sklearn.metrics.pairwise.distance_metrics
  knn_classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', weights='distance')
  knn_classifier.fit(x_train_pca, y_train_labels)

  # Тестирование

  # Извлекаем признаки из изображений тестовой выборки test_ds
  x_test_features, y_test_labels = extract_features(test_ds, model)

  # Применение созданного ранее PCA для тестовых данных
  x_test_pca = pca.transform(x_test_features)

  # Классификация тестовых изображений с использованием KNN
  y_pred = knn_classifier.predict(x_test_pca)

  # Рассчёт точности
  accuracy = accuracy_score(y_test_labels, y_pred)
  print("CNN + PCA + KNN accuracy:", accuracy)

train_and_test_CNN_PCA_KNN(x_train_features, y_train_labels)

1/1 [==============================] - 1s 1s/step
CNN + PCA + KNN accuracy: 0.8483333333333334
